In [2]:
from graphviz import Digraph
from collections import defaultdict
from Bio import SeqIO
import time

In [130]:
class Vertex:
    id = None
    in_edges = None
    out_edges = None
    label = None
    def __init__(self, label, id):
        self.in_edges = []
        self.out_edges = []
        self.label = label
        self.id = str(id)
class Edge:
    start = None
    end = None
    label = None
    coverage = None
    n_kmers = None # Сколько к-меров соответствуют ребру - нужно для правильного пересчета
    def __init__(self,start, end, label, cov, kmers):
        self.start = start
        self.end = end
        self.label = label
        self.coverage = cov
        self.n_kmers = kmers
    def __repr__(self):
        return "From " + self.start.label + " to " + self.end.label + " with cov " + str(self.coverage) + ", kmers " + str(self.n_kmers)

In [4]:
def get_rc(read):
    compl = {"A" : "T", "T" : "A", "G" : "C", "C" : "G"}
    res = []
    for l in read:
        res.append(compl[l])
    return ''.join(res[::-1])

In [126]:
class Graph:
    ver_dict = None # Содержит вершины
    k_mer_len = None
    ver_id = None
    def __init__(self, l):
        self.ver_dict = {}
        self.k_mer_len = l
        self.ver_id = 1
        
    def render(self, label_type):
        f = Digraph('de_bruijn_graph', filename='fsm.gv', strict = False)
        used = []
        for _,v in self.ver_dict.items():
            for edge in v.out_edges:                    
                label = ""
                if "label" in label_type:
                    label += "Label: " + edge.label + " "
                    
                if "cov" in label_type:
                    label += "Coverage: " + "{:.2f}".format(edge.coverage) + " "
                
                if "len" in label_type:
                    label += "Length: " + str(len(edge.label) - self.k_mer_len) + " "
#                 f.edge(edge.start.label, edge.end.label, label=label)
                f.edge("Vertex #" + edge.start.id, "Vertex #" + edge.end.id, label=label)

        return f
    
    def print_edges(self, path):
        used = defaultdict(bool)
        f = open(path, "w")
        labels = []
        id = 1
        for _,v in self.ver_dict.items():
            for edge in v.out_edges:
                flag = True
                for label in labels:
                    if edge.label == get_rc(label):
                        flag = False
                if not flag:
                    continue
                f.write("> " + str(id) + "\n")
                f.write(edge.label + "\n")
                labels.append(edge.label)
                id+=1
        
    
    def add_edge(self, start, end, label, cov, kmers):
        v = self.ver_dict.get(start)
        if not v:
            v = Vertex(start, self.ver_id)
            self.ver_id+=1
            self.ver_dict[start] = v
        for edge in v.out_edges:
            if edge.end.label == end and edge.label == label:
                edge.coverage += cov # Если ребро существует, увеличиваем покрытие
                return
        t = self.ver_dict.get(end)
        if not t:
            t = Vertex(end, self.ver_id)
            self.ver_id+=1
            self.ver_dict[end] = t
        e = Edge(v, t, label, cov, kmers) # Иначе создаем новое
        v.out_edges.append(e)
        t.in_edges.append(e)
    
    def delete_vertex(self, v):
        for edge in v.in_edges:
            edge.start.out_edges.remove(edge)
            edge.end.in_edges.remove(edge)
        for edge in v.out_edges:
            edge.start.out_edges.remove(edge)
            edge.end.in_edges.remove(edge)
        self.ver_dict[v.label] = None
#         del(v)
        
    def free_graph(self): # Освобождаем граф от лишних вершин - делаем это постфактум для удобства
        for name in list(self.ver_dict.keys()):
            if self.ver_dict[name] is None:
                del(self.ver_dict[name])
                
    def condensate(self):
        for _,v in self.ver_dict.items():
            if len(v.in_edges) == 1 and len(v.out_edges) == 1:
                
                in_edge = v.in_edges[0]
                out_edge = v.out_edges[0]
                start = in_edge.start
                end = out_edge.end
                l1 = in_edge.label
                l2 = out_edge.label
                t = len(v.label)
                new_label = l1+l2[t:]
                new_kmers = in_edge.n_kmers + out_edge.n_kmers # Пересчитываем среднее правильно
                new_cov = (in_edge.coverage*in_edge.n_kmers + out_edge.coverage*out_edge.n_kmers) / new_kmers
                self.add_edge(start.label, end.label, new_label, new_cov, new_kmers)
                self.delete_vertex(v)
        self.free_graph()
                
    def remove_dead_ends(self, cov_threshold):
        retry = True # Проходим несколько раз, чтобы не оставалось хвостов
        while retry:
            retry = False
            for _,v in self.ver_dict.items():
                if len(v.out_edges) == 0 or len(v.in_edges) == 0:
                    flag = True # Если хоть кто-то превосходит, то не удаляем
                    for edge in v.in_edges+v.out_edges:
                        if edge.coverage > cov_threshold:
                            flag = False
                            break
                    if flag:
                        retry = True
                        self.delete_vertex(v)
            self.free_graph()
        
    def remove_bad_edges(self, cov_threshold, len_threshold):
        for _,v in self.ver_dict.items():
            for edge in v.in_edges + v.out_edges:
                if edge.coverage < cov_threshold or len(edge.label) < len_threshold:
                    edge.start.out_edges.remove(edge)
                    edge.end.in_edges.remove(edge)

In [134]:
def process_file(path, k, output, cov_threshold, format = "fastq", len_threshold = 0): # Кажется нецелесообразным выкидывать из-за длины
    print("Assembly started...")
    st_time = time.time()
    g = Graph(k)
    for record in SeqIO.parse(path, format):
        read = str(record.seq)
        for i in range(len(read)-k):
            start = read[i:i+k]
            end = read[i+1:i+k+1]
            label = read[i:i+k+1]
            g.add_edge(start, end, label, 1, 1)
        read = get_rc(read)
        for i in range(len(read)-k):
            start = read[i:i+k]
            end = read[i+1:i+k+1]
            label = read[i:i+k+1]
            g.add_edge(start, end, label, 1, 1)
    g.condensate()
    g.remove_dead_ends(30)
    g.condensate()
    
    if cov_threshold != -1:
        g.remove_bad_edges(cov_threshold, len_threshold)
        g.condensate()
    f = open(output+".dot", "w")
    f.write(g.render("len cov").source)
    g.print_edges(output + "_edges.fasta")
    print("Finished in", time.time() - st_time, "seconds.")

In [275]:
process_file("./5/ECOLI_IS220_QUAKE_1K_paired_reads.fasta", 12,"test", 1, format = "fasta")

Assembly started...
Finished in 0.3442704677581787 seconds.


In [135]:
process_file("./5/s_6.first1000.fastq", 55, "out1k", 0)

Assembly started...
Finished in 0.43497490882873535 seconds.


In [136]:
process_file("./5/s_6.first10000.fastq", 55, "out10k", 100)

Assembly started...
Finished in 7.235232591629028 seconds.


In [137]:
process_file("./5/s_6.first100000.fastq", 55, "out100k", 100)

Assembly started...
Finished in 74.4377064704895 seconds.


In [138]:
process_file("./5/s_6.first100000.fastq", 55, "out100k_with_bad_edges", -1) # Отбрасываем только хвостики

Assembly started...
Finished in 77.79249048233032 seconds.


В целом кажется, что отбрасывание всех плохих ребер эффективнее, так как отбрасывание хвостиков по сути является тем же самым, но с ограничениями. В итоге остается большое число неоднозначностей, как можно видеть на картинке.   
P.S. С удалением комплементарных последовательностей возникли проблемы - нельзя просто удалять, если мы уже добавили комплементарную - нам не известен порядок обхода. Из-за этого мы можем добавить первым не ту вершину. Можно было бы обходить деревья и выделять компоненты связности, но при данном задании графа это неоправданно трудозатратно.